#### Consider the flight dataset attached. Write a Python code block to find all the travel options a passenger can take, along with flight details for the input Delhi (origin) to Mumbai (destination).

In [1]:
from collections import defaultdict, deque

import pandas as pd

origin = " Delhi"
destination = " Mumbai"

flights_df = pd.read_csv("./Flight Details.csv")

In [2]:
def format_time(minutes):
    hours = minutes // 100
    mins = minutes % 100
    return f"{hours:02}:{mins:02}"


def calculate_total_duration(route):
    start_time = route[0]["StartTime"]
    end_time = route[-1]["EndTime"]
    duration = (end_time // 100 - start_time // 100) * 60 + (
        end_time % 100 - start_time % 100
    )
    return duration


def format_route(route):
    stops = len(route) - 1
    stop_details = (
        "Direct"
        if stops == 0
        else f"{stops} Stop(s) ({', '.join([f['Destination'] for f in route[:-1]])})"
    )
    total_duration = calculate_total_duration(route)
    return f"{route[0]['Origin'].strip()} - {route[-1]['Destination'].strip()} >> {total_duration // 60} hours {total_duration % 60} mins >> {stop_details}"

**Without using graph**

In [ ]:
def find_travel_options(flights, origin, destination):
    flights = flights.to_dict(orient="records")
    results = []

    def find_routes(current_route, current_flight):
        # Add current flight to the route
        current_route.append(current_flight)

        # Check if we reached the destination
        if current_flight["Destination"].strip() == destination:
            results.append(list(current_route))
            current_route.pop()  # Backtrack for other options
            return

        # Find connecting flights
        for flight in flights:
            if (
                flight["Origin"].strip() == current_flight["Destination"].strip()
                and flight["StartTime"] > current_flight["EndTime"]
            ):
                find_routes(current_route, flight)

        # Backtrack
        current_route.pop()

    # Find all starting flights from the origin
    for flight in flights:
        if flight["Origin"].strip() == origin:
            find_routes([], flight)

    return results


# Inputs
origin = "Delhi"
destination = "Mumbai"

# Get travel options
travel_options = find_travel_options(flights_df, origin, destination)

# Display travel options
if travel_options:
    for i, option in enumerate(travel_options, start=1):
        print(f"Option {i}:\n\t{format_route(option)}")
        for flight in option:
            print(
                f"\tFlight {flight['FlightNumber']} from {flight['Origin'].strip()} to {flight['Destination'].strip()} ({format_time(flight['StartTime'])} - {format_time(flight['EndTime'])})"
            )
else:
    print(f"No travel options available from {origin} to {destination}.")

**Using graph**

In [ ]:
def find_routes(origin, destination, flights):
    # Build a graph of routes
    graph = defaultdict(list)
    for _, row in flights.iterrows():
        graph[row["Origin"].strip()].append(
            {
                "FlightNumber": row["FlightNumber"],
                "Destination": row["Destination"].strip(),
                "Origin": row["Origin"].strip(),
                "StartTime": row["StartTime"],
                "EndTime": row["EndTime"],
            }
        )

    results = []
    queue = deque([(origin, [], 0)])

    while queue:
        current_city, path, last_end_time = queue.popleft()
        if current_city == destination:
            results.append(path)
            continue

        for flight in graph[current_city]:
            if (
                flight["StartTime"] >= last_end_time
            ):  # Check for valid connection timing
                queue.append(
                    (flight["Destination"].strip(), path + [flight], flight["EndTime"])
                )

    return results


# Find routes from Delhi to Mumbai
origin = "Delhi"
destination = "Mumbai"
travel_options = find_routes(origin, destination, flights_df)

# Display travel options
if travel_options:
    for i, option in enumerate(travel_options, start=1):
        print(f"Option {i}:\n\t{format_route(option)}")
        for flight in option:
            print(
                f"\tFlight {flight['FlightNumber']} from {flight['Origin'].strip()} to {flight['Destination'].strip()} ({format_time(flight['StartTime'])} - {format_time(flight['EndTime'])})"
            )
else:
    print(f"No travel options available from {origin} to {destination}.")